In [80]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset

raw_dataset = load_dataset("json", data_files="./movesets-train.json")

Generating train split: 467 examples [00:00, 16802.83 examples/s]


In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 467
    })
})

In [25]:
raw_dataset = raw_dataset.rename_column("output_text", "labels")
raw_dataset = raw_dataset.rename_column("input_text", "text")

In [26]:
raw_dataset["train"]

Dataset({
    features: ['text', 'labels'],
    num_rows: 467
})

In [75]:
def tokenize_data(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

In [76]:
tokenized_dataset = raw_dataset.map(tokenize_data, batched=True)

Map: 100%|██████████| 467/467 [00:00<00:00, 10441.55 examples/s]


In [78]:
# tokenized_dataset.rename_column("output_text", "labels")
# tokenized_dataset.rename_column("input_text", "text")
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

In [79]:
tokenized_dataset["train"][0]

{'labels': 'physical',
 'input_ids': [101,
  2054,
  2828,
  1997,
  2693,
  3503,
  2023,
  6412,
  1024,
  1996,
  5310,
  9530,
  14876,
  26698,
  1996,
  22277,
  2007,
  3177,
  1010,
  2059,
  18296,
  2229,
  1012,
  1996,
  2886,
  4915,
  2302,
  8246,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [72]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [81]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments("first-try-poke")

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

In [82]:
trainer.train()

  0%|          | 0/177 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).